In [ ]:
import os
import sys
import seaborn as sns
from time import sleep

## PCAP tests + graphs

In [ ]:

path = sys.argv[1]
path = 'D:/FINALQUICRESULTS/h3results'


pcap_file_sizes_grouped = {fn:[] for fn in list(map(lambda x: x.split('_')[1], os.listdir(path))) if fn.endswith('.pcap')}
pcaps = {fn:[] for fn in list(map(lambda x: x.split('_')[1], os.listdir(path))) if fn.endswith('.pcap')}



for f in os.listdir(path):
    filepath = os.path.join(path, f)
    if f.endswith('.pcap'):
        #pcap_filenames.append(f)
        # returns file size in bytes
        pcap_size = os.path.getsize(filepath)
        pcap_file_sizes_grouped[f.split('_')[1]].append(pcap_size)
        pcaps[f.split('_')[1]].append((f, pcap_size))
        
print(pcap_file_sizes_grouped)
print(pcap_file_sizes_grouped.keys())
        

In [ ]:

import os
h2domains = [fn for fn in list(map(lambda x: x.split('_')[1], os.listdir("D:/FINALH2RESULTS/h2results"))) if fn.endswith('.pcap')]
print(len(set(h2domains)))
h3domains = [fn for fn in list(map(lambda x: x.split('_')[1], os.listdir("D:/FINALQUICRESULTS/h3results"))) if fn.endswith('.pcap')]
print(len(set(h3domains)))
print(set(h3domains).symmetric_difference(set(h2domains)))

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# boxplots for all pcaps
# sorting by mean so it looks nicer
yss = list(pcap_file_sizes_grouped.values())

means = [sum(yss[i]) / len(yss[i]) for i in range(len(pcap_file_sizes_grouped))]
yss_zip = sorted(list(zip(means, yss)))
xss_zip = sorted(list(zip(means, list(pcap_file_sizes_grouped.keys()))))
yss_sorted_by_mean = list(map(lambda x: x[1], yss_zip))
xss_sorted_by_mean = list(map(lambda x: x[1], xss_zip))
xss_sorted_by_mean = list(map(lambda x: x[:-5], xss_sorted_by_mean))
print(yss_sorted_by_mean)


# printing outlier file names:
print(pcaps)
outliers={site:[] for site in pcaps.keys()}
no_of_outliers={site:0 for site in pcaps.keys()}
for k in pcaps.keys():
    ps = pcaps[k]
    q1 = np.percentile(list(map(lambda x: x[1], pcaps[k])), 25, method='midpoint')
    q3 = np.percentile(list(map(lambda x: x[1], pcaps[k])), 75, method='midpoint')
    iqr = q3 - q1
    upper_bound = q3 + (1.5 * iqr)
    lower_bound = q1 - (1.5 * iqr)
    for (fname, fsize) in ps:
        if fsize > upper_bound:
            print(f"{fname} is too large")
            outliers[k].append(fname)
            no_of_outliers[k] += 1
        elif fsize < lower_bound:
            print(f"{fname} is too small")
            outliers[k].append(fname)
            no_of_outliers[k] += 1
        
print()
for k in no_of_outliers:
    t=no_of_outliers[k]
    no_of_site_captures = len(pcaps[k])
    if t > 0 and t <= 10:
        print(f"{k} has {t} outliers")
        if no_of_site_captures - t >= 100:
            for o in outliers[k]:
                del_path=path + '/' + o
                #os.remove(del_path)
                print("outlier deleted")
        else:
            print("cannot safely delete outliers")
    elif t > 10:
        print(f"{k} has {t} outliers")
        
        print(f"{no_of_site_captures} - {t} = {no_of_site_captures-t} (if greater than 100 then safe to delete)")
        if no_of_site_captures-t >= 100:
            for o in outliers[k]:
                del_path=path + '/' + o
                #os.remove(del_path)
            print("outliers deleted")
        else:
            print("NOT A SAFE DELETE")
            
for k in pcaps:
    no_of_site_captures = len(pcaps[k])
    if no_of_site_captures < 100:
            print(f"ERROR: less than 100 captures for site {k}, pcaps left = {no_of_site_captures}")



In [ ]:
plt.boxplot(yss_sorted_by_mean[0:20], showfliers=False)
plt.ylim([0, 1500000])
plt.xticks(range(1, 21), xss_sorted_by_mean[0:20], rotation=90)
plt.xlabel("Domain")
plt.ylabel("Packet Capture Size (Bytes)")

In [ ]:

plt.boxplot(yss_sorted_by_mean[20:40], showfliers=False)
plt.ylim(0, 2500000)
plt.xticks(range(1, 21), xss_sorted_by_mean[20:40], rotation=90)
plt.xlabel("Domain")
plt.ylabel("Packet Capture Size (Bytes)")

In [ ]:
plt.boxplot(yss_sorted_by_mean[40:60], showfliers=False)
plt.xticks(range(1, 21), xss_sorted_by_mean[40:60], rotation=90)
plt.ylim(1000000, 3500000)
plt.xlabel("Domain")
plt.ylabel("Packet Capture Size (Bytes)")

In [ ]:
plt.boxplot(yss_sorted_by_mean[60:80])
plt.xticks(range(1, 21), xss_sorted_by_mean[60:80], rotation=90)
plt.ylim(2500000, 7000000)
plt.xlabel("Domain")
plt.ylabel("Packet Capture Size (Bytes)")

In [ ]:
plt.boxplot(yss_sorted_by_mean[80:])
plt.xticks(range(1, len(xss_sorted_by_mean[80:])+1), xss_sorted_by_mean[80:], rotation=90)
plt.ylim(5000000, 45000000)
plt.xlabel("Domain")
plt.ylabel("Packet Capture Size (Bytes)")

In [ ]:
# histogram of all data...
import seaborn as sns

listed_values = list(pcap_file_sizes_grouped.values())
flat_sizes = [v for i in range(len(listed_values)) for v in listed_values[i]]
print(flat_sizes)
sns.histplot(flat_sizes)

plt.xlabel("Packet Capture Size (Bytes)")

In [ ]:
print(len(pcap_file_sizes_grouped))
for k in pcap_file_sizes_grouped:
    print(k)

In [ ]:
# Checking for empty files
# NOTE: The pcap files will rarely be empty if broad filter was used when collecting traffic.
#       Better to filter to the relevant packets only then check if they are empty... 
#       But this can only be done on parsed traffic...

for k in pcap_file_sizes_grouped.keys():
    for f_size in pcap_file_sizes_grouped[k]:
        if f_size == 0:
            print(f"ERROR: one of the {k} pcaps is empty")

            


In [ ]:
# MEAN AND STANDARD DEVIATION FOR ALL CRAWLS TO A WEBSITE

from statistics import mean, stdev

# list_of_sites = ['ac', 'anu', 'berkeley', 'bristol', 'brown', 'bu', 'caltech',
#        'cam', 'ch', 'charite', 'cmu', 'columbia', 'cornell', 'cuhk',
#        'dartmouth', 'de', 'ed', 'edu', 'emory', 'epfl', 'eu', 'eur',
#        'gatech', 'gla', 'harvard', 'helsinki', 'hku', 'hu-berlin',
#        'imperial', 'jhu', 'kcl', 'kuleuven', 'kyoto-u', 'lmu', 'lse',
#        'lunduniversity', 'manchester', 'mcgill', 'mcmaster', 'mit',
#        'monash', 'northwestern', 'ntu', 'nyu', 'osu', 'ox', 'pku',
#        'princeton', 'psu', 'purdue', 'rice', 'rug', 'rwth-aachen', 'se',
#        'skku', 'snu', 'sorbonne-universite', 'stanford', 'sydney',
#        'tsinghua', 'tudelft', 'tum', 'u-tokyo', 'ubc', 'ucdavis',
#        'uchicago', 'ucl', 'ucla', 'ucsb', 'umn', 'umontreal', 'unc',
#        'uni-heidelberg', 'unimelb', 'universiteitleiden', 'unsw', 'upenn',
#        'uq', 'usc', 'ustc', 'utexas', 'utoronto', 'uu', 'uva', 'uzh',
#        'washington', 'wisc', 'wur', 'yale']

# pcap_names = os.listdir(path)

pcap_filenames = []
pcap_filesizes = []

filesize_zipped = list(zip(pcap_filenames, pcap_filesizes))

sorting_filesize_zipped = sorted(list(map(lambda x: (x[0][2:], x[1]), filesize_zipped)))

# grouping the files by website
just_the_sizes_grouped = []
builder = []
temp = sorting_filesize_zipped[0][0]
for i in range(len(sorting_filesize_zipped)):
    (name, size) = sorting_filesize_zipped[i]
    if name != temp:
        temp = name
        just_the_sizes_grouped.append((temp, builder))
        builder = []
    builder.append(size)
    
    # if its the last element then push the file sizes anyway
    if sorting_filesize_zipped[-1] == sorting_filesize_zipped[i]:
        just_the_sizes_grouped.append((temp, builder))
    
print(just_the_sizes_grouped)

# calculate mean and std deviation of all crawls to a website
means = {}
std_devs = {}
for (name, size_list) in just_the_sizes_grouped:
    means[name] = mean(size_list)
    std_devs[name] = stdev(size_list)
    # outlier threshold = mean +/- 3 std devs
    lower_threshold = means[name] - 3*std_devs[name]
    upper_threshold = means[name] + 3*std_devs[name]
    for s in size_list:
        if s < lower_threshold or s > upper_threshold:
            print(f"outlier found in {name}: value = {s}")
            
    


In [ ]:
from statistics import mean, stdev

means = {}
std_devs = {}
for name in pcap_file_sizes_grouped:
    size_list = pcap_file_sizes_grouped[name]
    means[name] = mean(size_list)
    std_devs[name] = stdev(size_list)
    # outlier threshold = mean +/- 3 std devs
    lower_threshold = means[name] - 3*std_devs[name]
    upper_threshold = means[name] + 3*std_devs[name]
    for s in size_list:
        if s < lower_threshold:
            print(f"outlier found in {name}: value = {s} (big)")
        elif s > upper_threshold:
            print(f"outlier found in {name}: value = {s} (small)")

In [ ]:
# check screenshots

from PIL import ImageChops
import math, operator

def rmsdiff(im1, im2):
    "Calculate the root-mean-square difference between two images"

    h = ImageChops.difference(im1, im2).histogram()

    # calculate rms
    return math.sqrt(reduce(operator.add,
        map(lambda h, i: h*(i**2), h, range(256))
    ) / (float(im1.size[0]) * im1.size[1]))
    
    
# deprecated, just checking by eye instead.
    

## Parsed pcap tests

In [ ]:
# finding the smallest and largest packets
# find the number of packets
# find the mean packet size
# print the ratio of incoming to outgoing 
# check there are actually incoming and outgoing packets

import pandas as pd
import numpy as np
import os

# could store these stats so they are easier to compare to other captures of the same website

# could label df to make a little easier to read
cols = {'capture_name':[], 'smallest_pkt':[], 'largest_pkt':[], 'mean_pkt':[], 'ratio':[], 'negative':[]}
pkt_tests = pd.DataFrame(cols)

root_path = 'C:/Users/andre/Uni_Documents/Dissertation/website-fingerprinting/parsed_results/h3results/parsed-h3-results'

for f in os.listdir(root_path):
    largest_pkt_size = 0
    smallest_pkt_size = 99999
    fpath = os.path.join(root_path, f)
    current_capture = pd.read_csv(fpath, sep='\t')
    current_cap_pkt_sizes = []
    positive = 1
    negative = 1
    print("current file:", f)
    for index, row in current_capture.iterrows():
        if row[3] == 2020 or row[4] == 2020 or row[5] == 2020 or row[6] == 2020:
            pkt_size = row[10]
            current_cap_pkt_sizes.append(pkt_size)
            if pkt_size < smallest_pkt_size:
                smallest_pkt_size = pkt_size
            if pkt_size > largest_pkt_size:
                largest_pkt_size = pkt_size
                
            if row[4] == 2020 or row[6] == 2020:
                positive += 1
            elif row[3] == 2020 or row[5] == 2020:
                negative += 1
    print(f"capture {f}: smallest={smallest_pkt_size}, largest={largest_pkt_size}, mean size={np.mean(current_cap_pkt_sizes)}, ratio={positive / negative}, -ve={negative-1}")
    pkt_tests.loc[-1] = [f, smallest_pkt_size, largest_pkt_size, np.mean(current_cap_pkt_sizes), positive/negative, negative-1]
    pkt_tests.index = pkt_tests.index + 1
        
        
    


In [ ]:
sus_caps = pkt_tests[pkt_tests['negative'] == 0]
sus_caps

In [ ]:
# stats for all collected pkts:
print(f"smallest pkt = {pkt_tests['smallest_pkt'].min()}")
print(f"largest pkt = {pkt_tests['largest_pkt'].max()}")
print(f"mean pkt = {pkt_tests['mean_pkt'].mean()}")


In [ ]:
# finding the smallest and largest packets
# find the number of packets
# find the mean packet size
# print the ratio of incoming to outgoing 
# check there are actually incoming and outgoing packets

import pandas as pd
import numpy as np
import os

# could store these stats so they are easier to compare to other captures of the same website

# could label df to make a little easier to read
cols = {'capture_name':[], 'smallest_pkt':[], 'largest_pkt':[], 'mean_pkt':[], 'ratio':[], 'negative':[]}
pkt_tests = pd.DataFrame(cols)

root_path = 'D:/FINALH2RESULTS/parsed-h2-results'

for f in os.listdir(root_path):
    largest_pkt_size = 0
    smallest_pkt_size = 99999
    fpath = os.path.join(root_path, f)
    current_capture = pd.read_csv(fpath, sep='\t')
    current_cap_pkt_sizes = []
    positive = 1
    negative = 1
    print("current file:", f)
    for index, row in current_capture.iterrows():
        if row[3] == 2020 or row[4] == 2020 or row[5] == 2020 or row[6] == 2020:
            pkt_size = row[12]
            current_cap_pkt_sizes.append(pkt_size)
            if pkt_size < smallest_pkt_size:
                smallest_pkt_size = pkt_size
            if pkt_size > largest_pkt_size:
                largest_pkt_size = pkt_size
                
            if row[4] == 2020 or row[6] == 2020:
                positive += 1
            elif row[3] == 2020 or row[5] == 2020:
                negative += 1
    print(f"capture {f}: smallest={smallest_pkt_size}, largest={largest_pkt_size}, mean size={np.mean(current_cap_pkt_sizes)}, ratio={positive / negative}, -ve={negative-1}")
    pkt_tests.loc[-1] = [f, smallest_pkt_size, largest_pkt_size, np.mean(current_cap_pkt_sizes), positive/negative, negative-1]
    pkt_tests.index = pkt_tests.index + 1
        
        
    
print(f"smallest pkt = {pkt_tests['smallest_pkt'].min()}")
print(f"largest pkt = {pkt_tests['largest_pkt'].max()}")
print(f"mean pkt = {pkt_tests['mean_pkt'].mean()}")

In [ ]:
pkt_tests.sort_values(by=['negative']).tail(20)

In [ ]:
# Are there any missed sites? yes usually...
import os
import sys
target_sites = []
crawled_sites = []
for f in os.listdir("C:/Users/andre/Uni_Documents/Dissertation/website-fingerprinting/parsed_results/h3results/parsed-h3-results"):
    crawled_sites.append(f.split('_')[1].split('.')[0])
crawled_sites = list(set(crawled_sites))
print(f"crawled {len(crawled_sites)} sites")

for s in sorted(crawled_sites):
    print(s)


# SOME URLS W/O www. in their url have failed their crawl due to the way I name output pcaps, these will need to be recrawled...
# A list of these is: (sites with a ~ were probs saved but not saved as the right name)
# 1- ethz.ch
# 2- umich.edu
# 3- duke.edu
# 4- nus.edu.sg
# 5- ucsd.edu
# 6- ki.se 
# 7- kust.edu.uk ~
# 8- psl.eu ~
# 9- illinois.edu
# 10- wustl.edu
# 11- en.snu.ac.kr ~ -> nah this one is good
# 12- win-cities.umn.edu ~
# 13- uni-freiburg.de
# 14- warwick.ac.uk ~
# 15- umd.edu 
# 16- uni-teubingen.de ~
# 17- uci.edu
# 18- msu.edu

# need to delete: edu, eu, ac, de, ch, umn, and replace with these newly crawled sites


